### Task 2: Use the converted weights and compare the fine-tuning results on CIFAR-10 dataset.
* Training from scratch
* ImageNet weights
* SimCLR pre-trained model 

In [2]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

#########################################
## TODO: 
## 1. create 3 instances of resnet50 model:
##   - The model with no pre-trained weights
##   - The model with pre-trained weights on ImageNet
##   - The model with SimCLR pre-trained weights
## 2. Replace the last layer to have 10 classes (CIFAR-10)
## 3. Train the models and compare the performance
## 4. Experiment with freezing/unfreezing the layers on the pre-trained models
## 5. Add the validation and testing steps
#########################################
model = models.resnet50(pretrained=True)

sd = torch.load('resnet50-1x.pth')
model.load_state_dict(sd['state_dict'])



print (model)
model.fc = nn.Linear(2048, 10)

transform = transforms.Compose([ transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
model = model.cuda()
dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
print (len(dataset))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True, num_workers=4)

model.train()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.1, momentum=0.9, weight_decay=1e-4)

for epoch in range(10):
    loss_train = 0
    for i, (input, target) in enumerate(dataloader):
        input = input.cuda()
        target = target.cuda()

        output = model(input)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
    print (epoch, loss_train/len(dataloader))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 